# Purpose

The purpose of this application is to publish the YAML formatted documents

Output formats:

* Pandoc
* HTML
* <s>PDF</s>
* CommonMark

# Discussion


References

* [PyYAML](https://pyyaml.org/)
* [pandoc](https://pandoc.org/)
  * [TeX Live](https://www.tug.org/texlive/) which is needed to [create a PDF](https://pandoc.org/MANUAL.html#creating-a-pdf)

# Install

Anything that needs installed to use this should be listed here

``` shell
#PyYAML
sudo -H pip3 install pyyaml

#Pandoc
sudo apt install pandoc
sudo apt install texlive-latex-recommended
```

# Application

## Configuration

In [1]:
#these are the base dirs everthing works from:
srcDir="./src"
dstDir="./docs"

#The node names are based on the [Pandoc Supported Formats](https://github.com/tajmone/markdown-guide/tree/master/pandoc#pandoc-supported-formats)
#the `dstPath` likely could go away as it was based upon a different idea for how the files should be stored
#this is a dictionary of dictionaries.  To access data: `outputConfig['yaml']['ext']`
outputConfig = {
    'yaml': {
        'ext': '.yaml', 
        'dstPath': dstDir
    },
    'markdown': { #Pandoc MarkDown varient
        'ext': '.pandoc', 
        'dstPath': dstDir
    },
    'markdown_strict': { #CommonMark spec
        'ext': '.md', 
        'dstPath': dstDir
    },
    'html5': {
        'ext': '.html', 
        'dstPath': dstDir
    },
    'pdf': {
        'ext': '.pdf', 
        'dstPath': dstDir
    }
}
#outputConfig

## function definitions

### find()

In [2]:
#find files that match the pattern

#these functions came from:
#https://stackoverflow.com/questions/1724693/find-a-file-in-python

"""
#this will find the first match
import os
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

#this will find all matches
import os
def find_all(name, path):
    result = []
    for root, dirs, files in os.walk(path):
        if name in files:
            result.append(os.path.join(root, name))
    return result
"""

#this will match a pattern
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result
#find('*.txt', '/path/to/dir')

### file operations

#### mkdir_p() and safe_open_w()

In [3]:
#the following is from:
#https://stackoverflow.com/questions/23793987/write-file-to-a-directory-that-doesnt-exist

import os, os.path
import errno

# Taken from https://stackoverflow.com/a/600612/119527
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def safe_open_w(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    mkdir_p(os.path.dirname(path))
    return open(path, 'w')

#with safe_open_w('/Users/bill/output/output-text.txt') as f:
#    f.write(...)

### yaml2python()

In [4]:
#this will convert a yaml file to a python data structure.

import yaml

def yaml2python (file):
    #print (file)

    f = open (file, 'r') #open the file to read
    content = yaml.load(f.read(), Loader=yaml.FullLoader) #convert the yaml contents into a python data structure
    f.close() #close the file
    
    #print (content)
    
    return content

### generateDocumentBody()

In [5]:
#walk and print the tree within documentBody

#this whole thing needs worked on and is mostly psudocode

def generateDocumentBody (outputData, srcData):
    if "YAML structure":
        for element in srcData: #walk the structure
            if "a .md file":
                #print contents
                pass
            elif "a .yaml file": #this should be an ID.  find() the corresponding .yaml file
                content = yaml2python(file)
                outputData += generateDocumentBody(outputData, srcData)
    else: #its just MD text
        outputData += srcData
    
    return (outputData)

### generatePandocContents()

In [6]:
#the pandoc file is the 'universal' format that all others will be generated from
#`data` = the full python data structure object

def generatePandocContents (data):
    document = "" #start with a blank document
    
    #################################################
    #Pandoc’s Markdown [Metadata blocks](https://pandoc.org/MANUAL.html#metadata-blocks)
    #Extension: pandoc_title_block
    
    keys=[
        "title",
        "author" #pretending that author will never be more then 1 entry
    ]
    for key in keys:
        if data.get(key) != None: #from [here](https://thispointer.com/python-how-to-check-if-a-key-exists-in-dictionary/)
            document += '% ' + data[key] + '\n'
    
    date=""
    for element in data['revision']:  #step through the list elements
        #print (element['date'])
        if date < element['date']: #REPLACE THIS with something that understands full ISO date/time formatted data!
            date = element['date']
    document += '% ' + date + '\n' #output the final result
    
    #################################################
    #Pandoc’s Markdown [Metadata blocks](https://pandoc.org/MANUAL.html#metadata-blocks)
    #Extension: yaml_metadata_block
    
    document += "\n" #pandoc expects a blank line before this section
    document += "---\n" #this is required to start the yaml metadata block
    #dump the following info:
    keys=[
        "title",
        "author",
        "keywords",
        "abstract",
        "lang"
    ]
    for key in keys:
        if data.get(key) != None: #from [here](https://thispointer.com/python-how-to-check-if-a-key-exists-in-dictionary/)
            document += '"' + key + '": ' + yaml.dump(data[key], default_style='"') #output format: `"key": "value"\n`
    
    document += "...\n\n" #this is required to end the yaml metadata block

    #################################################
    
    #document += generateDocumentBody (document, data['documentBody']) #work through the possible formats and correctly print
    document += data['documentBody']
    document += "\n"
    
    #################################################
    
    #print the revisions as a table in HTML because CommonMark doesnt do that
    document += "---\n\n"
    document += "# Revision history\n\n"
    
    #table style
    document += "<style>\n"
    document += "table { border-collapse: collapse; width: 100%; }\n"
    document += "td, th { border: 1px solid #dddddd; text-align: left; padding: 8px; }\n"
    document += "</style>\n"
    document += "\n"
    
    #Table layout:  "date", "status", "name", "reason"
    document += "<table>\n"
    document += "<tr>\n"
    document += "<th>Date</th>\n"
    document += "<th>Name</th>\n"
    document += "<th>Reason</th>\n"
    document += "</tr>\n"
    #the 'revisions' key consists of a list of sets
    #`content['revision'][0]['date']` will provide that specific data element
    for element in data['revision']:  #step through the list elements
        document += "<tr>\n"
        #now print the fields
        document += "<td>" + element['date'] + "</td>\n"
        document += "<td>" + element['name'] + "</td>\n"
        document += "<td>" + element['reason'] + "</td>\n"
        document += "</tr>\n"
    document += "</table>\n"
    
    #################################################
    #we are done generating the document contents
    
    #print (document)
    return (document)    

### convert2format()

In [7]:
#convert the pandoc formatted text into the specified format

import subprocess

def convert2format(srcFile, dstFile, outputFormat):
    if (outputFormat == 'html5') or (outputFormat == 'markdown_strict'):
        #subprocess.run(['pandoc', content['id']+'.pandoc', '--html-q-tags', '-s', '-o', content['id']+'.html'])
        subprocess.run(['pandoc', srcFile, '-s', '--html-q-tags', '-f', 'markdown+yaml_metadata_block+pandoc_title_block', '-t', outputFormat, '-o', dstFile])
    elif outputFormat == 'pdf':
        pass #the pdf conversion is generating font errors
        #print ('Error:  PDF format is currently broken')
        #subprocess.run(['pandoc', srcFile, '-s', '--html-q-tags', '-t', outputFormat, '-o', dstFile])
    elif (outputFormat != 'markdown') and (outputFormat != 'yaml'): #error on all but these which we will quietly ignore
        print ('Error:  ' + outputFormat + ' is not supported')

## main program

### find the files that need to be processed

In [8]:
#look for all the yaml files

srcFiles = find('*' + outputConfig['yaml']['ext'], srcDir)
#srcFiles

### process each file

For simplicity, this is using the premise that each YAML document is compiled to an output dir based upon its ID.  The reason for this is to avoid orphan files from accumulating.  At some later point, the process should look for only what changed and adjust the specific destination dir accordingly.

This is the document summary data structure:

``` python
summaryDataDemo = {
    'type': { #`type` field: key index
        '1928374': { #`id` field: key index
            'title': 'text',
            'shortDescription': 'text',
            'url': {
                'html5': 'http://',
                'pdf': 'http://',
                'yaml': 'http://',
                'markdown': 'http://',
                'markdown_strict': 'http://'
            }
        },
        '9898472': { #`id` field: key index
            'title': 'text',
            'shortDescription': 'text',
            'url': {
                'html5': 'http://',
                'pdf': 'http://',
                'yaml': 'http://',
                'markdown': 'http://',
                'markdown_strict': 'http://'
            }
        }
    }
}
summaryDataDemo['type']['1928374']['url']['html5']
```

In [9]:
#delete the output dir tree before regenerating

summaryData = {} #start out with a blank dataset

#generate files
for file in srcFiles:
    print (file)

    #read the file and convert it to a python data structure
    content = yaml2python (file)
    #print (content)
    
    #################################################
    
    #one dir structure per file
    outputDir = outputConfig['markdown']['dstPath'] + "/" + content['id']
    pandocFile = outputDir + "/" + content['id'] + outputConfig['markdown']['ext']
 
    pandocContents = generatePandocContents (content)
    #print (output)
       
    #save the document to file
    f = safe_open_w(pandocFile) #create the path if needed and open a file for output
    f.write(pandocContents) #write the data to file
    f.close()  #close the file
    
    #do the format conversions
    for outputFormat in outputConfig:
        
        #construct the full destination file/path
        outputFile = outputDir + "/" + content['id'] + outputConfig[outputFormat]['ext']
        #print (outputFile)
        
        convert2format(pandocFile, outputFile, outputFormat)
        
    #################################################
    #save data that will be used for the summary    

    if summaryData.get(content['type']) == None:
        summaryData.update({content['type'] : {}})

    if summaryData[content['type']].get(content['id']) == None:
        summaryData[content['type']].update({content['id'] : {}})

    if summaryData[content['type']][content['id']].get('title') == None:
        summaryData[content['type']][content['id']].update({'title' : content['title']})
        
    if summaryData[content['type']][content['id']].get('shortDescription') == None:
        summaryData[content['type']][content['id']].update({'shortDescription' : content['shortDescription']})
        
    if summaryData[content['type']][content['id']].get('url') == None:
        summaryData[content['type']][content['id']].update({'url' : {}})
    
    for outputFormat in outputConfig:
        if outputFormat == 'yaml':
            continue #skip to the next element in the loop
        
        #this is the relative url for the file
        url = '/Documentation/' + content['id'] + '/' + content['id'] + outputConfig[outputFormat]['ext']
        
        if summaryData[content['type']][content['id']]['url'].get(outputFormat) == None:
            summaryData[content['type']][content['id']]['url'].update({outputFormat : url})
    
#summaryData

./src/32f2e7dc-0d00-4bd9-b39f-9d5b562f8415.yaml
./src/ad9c7149-fa91-4e62-a86b-fdc4387840d1.yaml
./src/1e7c647e-93d7-455d-b5a5-fd7205ca1b14.yaml
./src/cd498dae-9287-4432-91a3-97adf1ea4dd6.yaml
./src/c54c285f-eeb4-4a42-815f-9ea0656265e2.yaml
./src/ebd4a2a1-1c57-4dfb-b13b-c02355a40d74.yaml
./src/4480c89d-1da0-4f18-9f8b-f0238333e69a.yaml
./src/93a0be83-8e5a-4a46-b264-218646b412ce.yaml
./src/.ipynb_checkpoints/ebd4a2a1-1c57-4dfb-b13b-c02355a40d74-checkpoint.yaml
./src/.ipynb_checkpoints/1e7c647e-93d7-455d-b5a5-fd7205ca1b14-checkpoint.yaml
./src/4a4d6ec7-4cce-45f8-aa79-df1764058818/4a4d6ec7-4cce-45f8-aa79-df1764058818.yaml
./src/4a4d6ec7-4cce-45f8-aa79-df1764058818/.ipynb_checkpoints/4a4d6ec7-4cce-45f8-aa79-df1764058818-checkpoint.yaml


### generate summary file

In [11]:
#group by doc type
#title ([YAML] [pandoc] [md] [html] [pdf]): shortDescription

document = "# Document index\n\n"

for el1 in summaryData: #type
    #print (el1)
    document += el1 + '\n\n'
    
    for el2 in summaryData[el1]: #id
        #print ("  " + el2)
        
        document += '* [' + summaryData[el1][el2]['title'] + '](' + summaryData[el1][el2]['url']['html5'] + ') '

        document += '[(Pandoc)](' + summaryData[el1][el2]['url']['markdown'] + ') '
        document += '[(MarkDown)](' + summaryData[el1][el2]['url']['markdown_strict'] + ') '
        
        if summaryData[el1][el2]['shortDescription'] != None:
            document += summaryData[el1][el2]['shortDescription']
            
        document += '\n'
        
    document += '\n'

#print (document)

#save the file
mdFile = dstDir + "/index.md"
f = safe_open_w(mdFile) #create the path if needed and open a file for output
f.write(document) #write the data to file
f.close()  #close the file

#convert to html
convert2format(mdFile, dstDir + "/index.html", 'html5')